In [74]:
import pandas as pd
import numpy as np

import lightgbm as lgb
import os
import glob

from sklearn.model_selection import train_test_split,StratifiedKFold, KFold
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder, TargetEncoder


# pandasの行列を省略しない
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [75]:
train_df = pd.read_csv("G:/VScode/data/jLeague/train.csv")
test_df = pd.read_csv("G:/VScode/data/jLeague/test.csv")
venue_df = pd.read_csv("G:/VScode/data/jLeague/venue_information.csv")
match_df = pd.read_csv("G:/VScode/data/jLeague/match_reports.csv")
holidays_df = pd.read_csv("G:/VScode/data/jLeague/holidays_in_japan.csv")
sampel_df = pd.read_csv("G:/VScode/data/jLeague/sample_submit.csv", header=None)

In [76]:
train_df.head()

,id,match_date,kick_off_time,section,round,home_team,away_team,venue,weather,temperature,humidity,broadcasters,attendance
0,9190,2006-03-04,16:04,第1節,第1日,G大阪,浦和,万博記念競技場,晴,8.3,40,NHK総合/J SPORTS(録),20916
1,9191,2006-03-05,13:00,第1節,第2日,甲府,清水,山梨県小瀬スポーツ公園陸上競技場,晴,12.9,28,山梨放送/テレビ静岡(録)/J SPORTS(録),14277
2,9192,2006-03-05,13:35,第1節,第2日,FC東京,大分,味の素スタジアム,晴,12.1,35,BS-i/MXテレビ(録)/J SPORTS(録),22531
3,9193,2006-03-05,14:04,第1節,第2日,磐田,福岡,静岡スタジアムエコパ,晴,11.6,42,J SPORTS,28564
4,9194,2006-03-05,14:04,第1節,第2日,名古屋,C大阪,名古屋市瑞穂陸上競技場,晴,13.1,32,スカイパーフェクTV!/NHK名古屋(録)/NHK大阪(録)/J SPORTS(録),17199


In [77]:
# broadcastersの/区切りユニークリストを取得
broadcasters = []
for i in train_df["broadcasters"].unique():
    broadcasters.extend(i.split("/"))
broadcasters = list(set(broadcasters))

for problem in broadcasters:
    train_df[problem] = 0
    test_df[problem] = 0

train_df['problem_count'] = 0
test_df['problem_count'] = 0

# problemlistにあったらonehotする
for i in train_df.index:
    p_count = 0
    for problem in broadcasters:
        if(problem in train_df.loc[i, 'broadcasters']):
            train_df.loc[i, problem] = 1
    train_df.loc[i, 'problem_count'] = p_count

for i in test_df.index:
    p_count = 0
    for problem in broadcasters:
        if(problem in test_df.loc[i, 'broadcasters']):
            test_df.loc[i, problem] = 1
    test_df.loc[i, 'problem_count'] = p_count

C:\Users\koba_\AppData\Local\Temp\ipykernel_17696\3096486656.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_df[problem] = 0
C:\Users\koba_\AppData\Local\Temp\ipykernel_17696\3096486656.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_df[problem] = 0
C:\Users\koba_\AppData\Local\Temp\ipykernel_17696\3096486656.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.conca

In [78]:
def clean(df):
    # 祝日の付与
    holidays_df.rename(columns={'holiday_date': 'match_date'}, inplace=True)
    holidays_df['holiday_flag'] = 1
    df = pd.merge(df, holidays_df, on='match_date', how='left')
    df['holiday_flag'] = df['holiday_flag'].fillna(0)

    # match_dateの日付型に変換
    df['match_date'] = pd.to_datetime(df['match_date'])
    df['year'] = df['match_date'].dt.year
    df['month'] = df['match_date'].dt.month
    df['day'] = df['match_date'].dt.day
    df['day_name'] = df['match_date'].dt.day_name()

    # kickoff_timeの時間型に変換
    df['kick_off_time'] = pd.to_datetime(df['kick_off_time'], format='%H:%M')
    df['kick_off_time'] = df['kick_off_time'].dt.hour + df['kick_off_time'].dt.minute / 60

    # sectionの数値化
    df['section'] = df['section'].str.replace('第', '').str.replace('節', '').astype(int)

    # roundの数値化
    df['round'] = df['round'].str.replace('第', '').str.replace('日', '').astype(int)

    # 雨の付与
    df['rain'] = 0
    df.loc[df['weather'].str.contains('雨'), 'rain'] = 1

    # 晴れの付与
    df['sunny'] = 0
    df.loc[df['weather'].str.contains('晴'), 'sunny'] = 1

    # broadcastersの数値化
    df['broadcasters'] = df['broadcasters'].str.count('/') + 1

    # match_dfの結合
    df = pd.merge(df, match_df, on='id', how='left')
    
    # venueの結合
    df = pd.merge(df, venue_df, on='venue', how='left')

    return df

train_clean = clean(train_df)
test_clean = clean(test_df)
train_clean.head()


C:\Users\koba_\AppData\Local\Temp\ipykernel_17696\3441286472.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['year'] = df['match_date'].dt.year
C:\Users\koba_\AppData\Local\Temp\ipykernel_17696\3441286472.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['month'] = df['match_date'].dt.month
C:\Users\koba_\AppData\Local\Temp\ipykernel_17696\3441286472.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider join

,id,match_date,kick_off_time,section,round,home_team,away_team,venue,weather,temperature,humidity,broadcasters,attendance,MXテレビ(録),NHK津,NHK BS1(101ch),スカパー光(TBSチャンネル),BSーi,NHK名古屋(録),tvk,BS-i(録),tvk(録),BSーTBS,DAZN ※AFCチャンピオンズリーグ2017の影響により、G大阪vs鹿島は7月5日(水)に開催,J SPORTS ESPN,テレビ埼玉(録),NHK岐阜,ひかりTV,スカパー・e2(TBSチャンネル),北海道文化放送,福岡放送,福岡放送(録),TOKYO MX*,MXテレビ*,NHK-BSハイビジョン,中国放送,NHK BS1 ※4 AFCチャンピオンズリーグ2016の影響により、浦和vsFC東京は6月22日(水)に開催,スカパースカチャン,BSーTBS(録),スカパー・e2(J sporst Plus),NHK京都,北海道テレビ,名古屋テレビ,テレビ大分,スカパー!・e2(J sports Plus),NHK静岡(録),スカパー光(J Sports Plus),e2スカチャンHD,TOKYO MX(録),NHK BS1(102ch),信越放送,北海道テレビ放送,MXテレビ,スカパー・e2(J sports Plus),スカパー!e2(J sports Plus),J SPORTS,TBS(単),NHK甲府,e2(スカチャン!HV),山梨放送(録),テレビ北海道,スカパー光(J sports Plus),スカパー光,スカパー!,BS-i,静岡放送(録),新潟放送,NHK BS1 ※6 浦和がスルガ銀行チャンピオンシップ2017SAITAMAに出場のため、7月22日(土)に開催,中部日本放送(録),NHK大分,テレ玉,NHK長野,DAZN,NHK徳島,サンテレビ,札幌テレビ,大分朝日放送,NHK BS1(録),KBS京都,スカパー!プレミアムサービス ※2 AFCチャンピオンズリーグ2015の影響により、柏vs浦和、G大阪vs鹿島は6月3日(水)に開催,テレビ新潟,静岡朝日テレビ,千葉テレビ(録),NHK福島,サンテレビ(録),テレビ西日本(録),東北放送,スカパー光(J SPORTS 4),テレビ信州,スカパー!(パーフェクト チョイス),TBS(単・録),BS-TBS,e2,静岡朝日テレビ(録),新潟総合テレビ,テレビ静岡,スカパー・e2(J Sports Plus),e2(スカチャン!),スカパー(スカチャンHD、スカチャン),BS※,テレビ神奈川,NHK神戸(近畿ブロック),NHK BS1,テレビ大阪,DAZN ※AFCチャンピオンズリーグ2017の影響により、川崎Fvs浦和は7月5日(水)に開催,NHK青森,NHK広島,NHK総合,J SPORTS2,NHK大阪,スカパー・e2(フジテレビONE),NHK名古屋,NHK大阪(近畿ブロック),スカパー!プレミアムサービス ※ スルガ銀行チャンピオンシップ2015OSAKAのため、7月22日(水)に開催,KBS京都(録),TVQ九州放送(録),e2(フジテレビONE),日本テレビ(録),山梨放送,e2(フジテレビTWO),NHK福岡,NHK静岡,e2(サムライ),BS-TBS(録),ミヤギテレビ,e2スカチャン,スカパー!プレミアムサービス,RKB毎日放送,静岡放送,スカパー!プレミアムサービス※1,TVQ九州放送,北海道放送,TOKYO MX*(録),テレ玉(録),e2(FIGHTING TV サムライ),TOKYO MX ※2 AFCチャンピオンズリーグ2016の影響により、FC東京vs広島、G大阪vs浦和は6月15日(水)に開催,NHK札幌,NHK大阪(録),毎日放送(録),スカパー,スカパー!プレミアムサービス ※2 AFCチャンピオンズリーグ2016の影響により、FC東京vs広島、G大阪vs浦和は6月15日(水)に開催,NHK新潟*,スカイパーフェクTV!,NHK山形,BS,スカパー!・e2(TBSチャンネル),NHK水戸,新潟テレビ21,仙台放送,名古屋テレビ(録),札幌テレビ放送,NHK神戸,NHK新潟,NHK水戸*,朝日放送(録),千葉テレビ,関西テレビ(録),NHK岩手,J SPORTS(録),TOKYO MX2,長野朝日放送,NHK神戸(録),東海テレビ(録),TOKYO MX,中京テレビ(録),NHK仙台(東北ブロック),スカパー!プレミアムサービス ※ 第95回天皇杯全日本サッカー選手権大会1回戦のため、9月23日(水・祝)に開催,e2(スカチャン!)(録),RKB毎日放送(録),日本テレビ,テレビ埼玉,テレビ山梨,テレビ静岡(録),スカパー!プレミアムサービス ※3 AFCチャンピオンズリーグ2015の影響により、柏vsG大阪は6月23日(火)に開催,スカパー光(J SPORTS 1),静岡第一テレビ(録),TBS,NHK秋田,長野放送,NHK総合(録),九州朝日放送,スカパー光(スカイ・A sports+),スカパー!・e2(J sprots Plus),NHK仙台,NHK仙台 ※ AFCチャンピオンズリーグ2015準決勝の影響により、10月25日(日)に開催。,東日本放送,NHK佐賀,BS(録),problem_count,description,holiday_flag,year,month,day,day_name,rain,sunny,home_team_player11,home_team_player10,home_team_player9,home_team_player8,home_team_player7,home_team_player6,home_team_player5,home_team_player4,home_team_player3,home_team_player2,home_team_player1,home_team_score,away_team_score,away_team_player1,away_team_player2,away_team_player3,away_team_player4,away_team_player5,away_team_player6,away_team_player7,away_team_player8,away_team_player9,away_team_player10,away_team_player11,capacity,address
0,9190,2006-03-04,16.066667,1,1,G大阪,浦和,万博記念競技場,晴,8.3,40,2,20916,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0.0,2006,3,4,Saturday,0,1,9 マグノアウベス FW,8 フェルナンジーニョ FW,10 二川孝広 MF,7 遠藤保仁 MF,27 橋本英郎 MF,17 明神智和 MF,14 家長昭博 DF,6 山口智 DF,2 シジクレイ DF,21 加地亮 DF,22 藤ヶ谷陽介 GK,1,1,23 都築龍太 GK,2 坪井慶介 DF,4 田中マルクス闘莉王 DF,20 堀之内聖 DF,6 山田暢久 MF,17 長谷部誠 MF,13 鈴木啓太 MF,8 三都主アレサンドロ MF,18 小野伸二 MF,10 ポンテ FW,21 ワシントン FW,21000,大阪府吹田市千里万博公園5-2
1,9191,2006-03-05,13.000000,1,2,甲府,清水,山梨県小瀬スポーツ公園陸上競技場,晴,12.9,28,3,14277,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0.0,2006,3,5,Sunday,0,1,18 長谷川太郎 FW,16

In [79]:
drop_cols = ['match_date', 'day', 'address', 'id', 'description', 'home_team_player1', 'home_team_player2', 'home_team_player3', 'home_team_player4',
'home_team_player5', 'home_team_player6', 'home_team_player7', 'home_team_player8',
'home_team_player9', 'home_team_player10', 'home_team_player11', 'away_team_player1',
'away_team_player2', 'away_team_player3', 'away_team_player4', 'away_team_player5',
'away_team_player6', 'away_team_player7', 'away_team_player8', 'away_team_player9',
'away_team_player10', 'away_team_player11',]
ce_cols = ['venue', 'weather', 'day_name', 'holiday_flag',]
le_cols = []
te_cols = ['home_team', 'away_team',]

'''
'home_team_player1', 'home_team_player2', 'home_team_player3', 'home_team_player4',
'home_team_player5', 'home_team_player6', 'home_team_player7', 'home_team_player8',
'home_team_player9', 'home_team_player10', 'home_team_player11', 'away_team_player1',
'away_team_player2', 'away_team_player3', 'away_team_player4', 'away_team_player5',
'away_team_player6', 'away_team_player7', 'away_team_player8', 'away_team_player9',
'away_team_player10', 'away_team_player11',
'''

"\n'home_team_player1', 'home_team_player2', 'home_team_player3', 'home_team_player4',\n'home_team_player5', 'home_team_player6', 'home_team_player7', 'home_team_player8',\n'home_team_player9', 'home_team_player10', 'home_team_player11', 'away_team_player1',\n'away_team_player2', 'away_team_player3', 'away_team_player4', 'away_team_player5',\n'away_team_player6', 'away_team_player7', 'away_team_player8', 'away_team_player9',\n'away_team_player10', 'away_team_player11',\n"

In [80]:
print(len(train_clean[train_clean['year'] >= 2016]))

612


In [81]:
# カウントエンコーディング
for col in ce_cols:
    encoder = train_clean[col].value_counts()
    train_clean[col] = train_clean[col].map(encoder)
    test_clean[col] = test_clean[col].map(encoder)


# ターゲットエンコーディングパート
if(len(te_cols) > 0):
    target_enc = TargetEncoder(target_type='continuous', random_state=42)
    
    train_clean[te_cols] = target_enc.fit_transform(train_clean[te_cols], train_clean['attendance'])
    # テストデータにtransform
    test_clean[te_cols]  = target_enc.transform(test_clean[te_cols])

# ラベルエンコーディングパート
le = LabelEncoder()
# mergeしてfit, そのあとtransform
all_df = pd.concat([train_clean, test_clean], axis=0)
for column in le_cols:
    le.fit(all_df[column])
    test_clean[column] = le.transform(test_clean[column])
    train_clean[column] = le.transform(train_clean[column])

# 不要なカラムの削除
train_clean = train_clean.drop(drop_cols, axis=1)
test_clean = test_clean.drop(drop_cols, axis=1)
train_clean.head()

,kick_off_time,section,round,home_team,away_team,venue,weather,temperature,humidity,broadcasters,attendance,MXテレビ(録),NHK津,NHK BS1(101ch),スカパー光(TBSチャンネル),BSーi,NHK名古屋(録),tvk,BS-i(録),tvk(録),BSーTBS,DAZN ※AFCチャンピオンズリーグ2017の影響により、G大阪vs鹿島は7月5日(水)に開催,J SPORTS ESPN,テレビ埼玉(録),NHK岐阜,ひかりTV,スカパー・e2(TBSチャンネル),北海道文化放送,福岡放送,福岡放送(録),TOKYO MX*,MXテレビ*,NHK-BSハイビジョン,中国放送,NHK BS1 ※4 AFCチャンピオンズリーグ2016の影響により、浦和vsFC東京は6月22日(水)に開催,スカパースカチャン,BSーTBS(録),スカパー・e2(J sporst Plus),NHK京都,北海道テレビ,名古屋テレビ,テレビ大分,スカパー!・e2(J sports Plus),NHK静岡(録),スカパー光(J Sports Plus),e2スカチャンHD,TOKYO MX(録),NHK BS1(102ch),信越放送,北海道テレビ放送,MXテレビ,スカパー・e2(J sports Plus),スカパー!e2(J sports Plus),J SPORTS,TBS(単),NHK甲府,e2(スカチャン!HV),山梨放送(録),テレビ北海道,スカパー光(J sports Plus),スカパー光,スカパー!,BS-i,静岡放送(録),新潟放送,NHK BS1 ※6 浦和がスルガ銀行チャンピオンシップ2017SAITAMAに出場のため、7月22日(土)に開催,中部日本放送(録),NHK大分,テレ玉,NHK長野,DAZN,NHK徳島,サンテレビ,札幌テレビ,大分朝日放送,NHK BS1(録),KBS京都,スカパー!プレミアムサービス ※2 AFCチャンピオンズリーグ2015の影響により、柏vs浦和、G大阪vs鹿島は6月3日(水)に開催,テレビ新潟,静岡朝日テレビ,千葉テレビ(録),NHK福島,サンテレビ(録),テレビ西日本(録),東北放送,スカパー光(J SPORTS 4),テレビ信州,スカパー!(パーフェクト チョイス),TBS(単・録),BS-TBS,e2,静岡朝日テレビ(録),新潟総合テレビ,テレビ静岡,スカパー・e2(J Sports Plus),e2(スカチャン!),スカパー(スカチャンHD、スカチャン),BS※,テレビ神奈川,NHK神戸(近畿ブロック),NHK BS1,テレビ大阪,DAZN ※AFCチャンピオンズリーグ2017の影響により、川崎Fvs浦和は7月5日(水)に開催,NHK青森,NHK広島,NHK総合,J SPORTS2,NHK大阪,スカパー・e2(フジテレビONE),NHK名古屋,NHK大阪(近畿ブロック),スカパー!プレミアムサービス ※ スルガ銀行チャンピオンシップ2015OSAKAのため、7月22日(水)に開催,KBS京都(録),TVQ九州放送(録),e2(フジテレビONE),日本テレビ(録),山梨放送,e2(フジテレビTWO),NHK福岡,NHK静岡,e2(サムライ),BS-TBS(録),ミヤギテレビ,e2スカチャン,スカパー!プレミアムサービス,RKB毎日放送,静岡放送,スカパー!プレミアムサービス※1,TVQ九州放送,北海道放送,TOKYO MX*(録),テレ玉(録),e2(FIGHTING TV サムライ),TOKYO MX ※2 AFCチャンピオンズリーグ2016の影響により、FC東京vs広島、G大阪vs浦和は6月15日(水)に開催,NHK札幌,NHK大阪(録),毎日放送(録),スカパー,スカパー!プレミアムサービス ※2 AFCチャンピオンズリーグ2016の影響により、FC東京vs広島、G大阪vs浦和は6月15日(水)に開催,NHK新潟*,スカイパーフェクTV!,NHK山形,BS,スカパー!・e2(TBSチャンネル),NHK水戸,新潟テレビ21,仙台放送,名古屋テレビ(録),札幌テレビ放送,NHK神戸,NHK新潟,NHK水戸*,朝日放送(録),千葉テレビ,関西テレビ(録),NHK岩手,J SPORTS(録),TOKYO MX2,長野朝日放送,NHK神戸(録),東海テレビ(録),TOKYO MX,中京テレビ(録),NHK仙台(東北ブロック),スカパー!プレミアムサービス ※ 第95回天皇杯全日本サッカー選手権大会1回戦のため、9月23日(水・祝)に開催,e2(スカチャン!)(録),RKB毎日放送(録),日本テレビ,テレビ埼玉,テレビ山梨,テレビ静岡(録),スカパー!プレミアムサービス ※3 AFCチャンピオンズリーグ2015の影響により、柏vsG大阪は6月23日(火)に開催,スカパー光(J SPORTS 1),静岡第一テレビ(録),TBS,NHK秋田,長野放送,NHK総合(録),九州朝日放送,スカパー光(スカイ・A sports+),スカパー!・e2(J sprots Plus),NHK仙台,NHK仙台 ※ AFCチャンピオンズリーグ2015準決勝の影響により、10月25日(日)に開催。,東日本放送,NHK佐賀,BS(録),problem_count,holiday_flag,year,month,day_name,rain,sunny,home_team_score,away_team_score,capacity
0,16.066667,1,1,17313.500322,24665.349093,149,1828,8.3,40,2,20916,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3387,2006,3,2342,0,1,1,1,21000
1,13.000000,1,2,12101.380307,18564.704895,32,1828,12.9,28,3,14277,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3387,2006,3,783,0,1,0,2,15859
2,13.583333,1,2,25929.415471,15673.968827,186,1828,12.1,35,3,22531,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3387,2006,3,783,0,1,2,0,47851
3,14.066667,1,2,14194.902372,15226.961702,7,1828,11.6,42,1,28564,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [82]:
# モデルの学習

test_preds = []
valid_scores = []

# yであるattendanceを取り出し、train_cleanから削除
y = train_clean['attendance']
train_clean.drop('attendance', axis=1, inplace=True)

# trainデータとvalidデータに分割
# 後ろ612行をvalidデータとして使用
X_train = train_clean[train_clean['year'] < 2016]
X_valid = train_clean[train_clean['year'] >= 2016]
y_train = y[train_clean['year'] < 2016]
y_valid = y[train_clean['year'] >= 2016]

# X_train, X_valid, y_train, y_valid = train_test_split(train_clean, y, test_size=0.2, random_state=42, shuffle=False)

lgb_train = lgb.Dataset(X_train, y_train,)
lgb_eval = lgb.Dataset(X_valid, y_valid, )

verbose_eval = -1 
params = {'objective': 'regression',
        'metric': 'rmse',
        'seed': 0,
        'verbose': -1,
        'num_leaves': 31,
        'min_data_in_leaf':20,
        'learning_rate': 0.01,
        'feature_fraction': 0.30,
        'max_depth': 6,}    
model = lgb.train(params,
                lgb_train,
                valid_sets=[lgb_train, lgb_eval],
                num_boost_round=3000,
                callbacks=[lgb.early_stopping(stopping_rounds=100, 
                            # verbose=False,
                            ), 
                    lgb.log_evaluation(verbose_eval)],
                )

# testでのスコアを算出し、test_predsに格納
test_pred = model.predict(test_clean, num_iteration=model.best_iteration)
test_preds.append(test_pred)
valid_scores.append(model.best_score['valid_1']['rmse'])

# 予測値を格納
sampel_df[1] = test_pred
print(valid_scores)
# sampel_df.to_csv("G:/VScode/data/jLeague/subs/sub_1.csv", index=False, header=False)

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1315]	training's rmse: 2648.1	valid_1's rmse: 4944.88
[4944.8832662493605]


In [87]:
# 1325をbest_iterationとして全データで再学習
lgb_train = lgb.Dataset(train_clean, y,)

verbose_eval = -1 
params = {'objective': 'regression',
        'metric': 'rmse',
        'seed': 0,
        'verbose': -1,
        'num_leaves': 31,
        'min_data_in_leaf':20,
        'learning_rate': 0.01,
        'feature_fraction': 0.30,
        'max_depth': 6,}    
model = lgb.train(params,
                lgb_train,
                num_boost_round=1325,
                )

# testでのスコアを算出し、test_predsに格納
test_pred = model.predict(test_clean, num_iteration=model.best_iteration)
test_preds.append(test_pred)

# 予測値を格納
sampel_df[1] = test_pred
sampel_df.to_csv("G:/VScode/data/jLeague/subs/sub_1.csv", index=False, header=False)

In [84]:
sampel_df.head()

,0,1
0,20745,8887.012763
1,20746,35058.323609
2,20747,18154.282610
3,20748,19532.334667
4,20749,11636.603169


In [85]:
importance = model.feature_importance()
feature_names = model.feature_name()

df_importance = pd.DataFrame({'Feature': feature_names, 'Importance': importance})
# importtane順に並び替え
df_importance = df_importance.sort_values('Importance', ascending=False)
df_importance

,Feature,Importance
3,home_team,3323
4,away_team,2974
194,capacity,2814
1,section,2334
7,temperature,2332
5,venue,2306
8,humidity,2043
0,kick_off_time,1913
187,year,1647
188,month,1397
